# Beatriz Cárdaba Rico

## TAREA FINAL: Datos de vivienda

<p> El negocio inmobiliario es uno de los motores de la economía de España, el carácter turístico de nuesta península hace que sean muchos los inversores extranjeros que se decidan por buscar una inversión de bajo riesgos en nuestras ciudades costeras o grandes urbes. 
Muchas de estas inversiones se hacen sobre grandes bolsas de inmuebles que deben ser analizados previamente para comprobar la rentabilidad del porfolio </p>

<!-- <p> En este caso vamos a trabajar con una tabla que contienen información de distintos inmuebles repartidos por una zona específica, sus carácterísticas y su precio </p>  -->

Todas las cuestiones se deben realizar sobre el conjunto de casos que representen viviendas ubicadas en zonas residenciales **(alta, media y baja densidad)**

**MUY IMPORTANTE:** En las otras prácticas he detectado colaboraciones involucrando varias personas y he sido flexible aunque a algunos os lo he mencionado en las correcciones, porque al final el trabajo de analista de datos es un trabajo colaborativo. Sin embargo, este trabajo es individual, así que cuidado con las colaboraciones.

Las variables de las que se compone el dataset son:

|NOMBRE VARIABLE|DESCRIPTOR|VALORES|
| --- | --- | --- |
|Order|Variable de identificación|1 a 2930|
|MS Zoning|Zona de ubicación de la vivienda|"A rural, C comercial, FV residencial flotante, I industrial, RH residencial alta densidad, RL residencial baja densidad, RM residencial media densidad"|
|Lot Frontage|Longitud de la fachada en pies||
|Lot Area|Superficie de la vivienda en pies cuadrados||
|Land Contour|Contorno del terreno circundante|"Lvl llano, Bnk Tipo bancal, HLS Ladera, Low Depresión"|
|Land Slope|Tipo de pendiente de la vivienda|" Gtl pendiente suave, Mod pendiente moderada, Sev fuerte pendiente"|
|Overall Qual|Grado de calidad de materiales y acabado de la vivienda|De 1 (Muy pobre) a 10 (Excelente)|
|Year Built|Año de construccion de la vivienda||
|Year Remod/Add|Año de última reforma de la vivienda||
|Mas Vnr Type|Tipo de revestimiento exterior|" BrkCmn Ladrillo normal, BrkFace Ladrillo visto, CBlock Bloque de cemento, None Ninguna, Stone Piedra "|
|Exter Qual|Calidad de revestimiento exterior|"Ex Excelente,Gd Bueno,TA Media,Fa Justo"|
|Bsmt Cond|Estado general del sótano|"Ex Excelente, Gd Bueno, TA Media, Fa Justo, Po Pobre,Ss sin sótano"|
|Total Bsmt SF|Superficie del sótano en pies cuadrados|
|Heating QC|Calidad de la calefacción|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|Central Air|Aire acondicionado centralizado|"N No Y Sí"|
|Full Bath|Número de baños completo en planta||
|Half Bath|Número de aseos en planta||
|Bedroom AbvGr|Número de dormitorios en planta||
|Kitchen AbvGr|Número de cocinas en planta||
|Kitchen Qual|Calidad de cocinas|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre"|
|TotRms AbvGrd|Número total de habitaciones excluidos los cuartos de baño||
|Garage Cars|Número de plazas de garaje||
|Garage Area|Superficie del garaje|||
|Garage Cond|Estado del garaje|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Po Pobre,Sg sin garaje"|
|Pool Area|Superficie de la piscina en pies cuadrados|
|Pool QC|Calidad de la piscina|"Ex Excelente,Gd Bueno,TA Media,Fa Justo,Sp no hay piscina"|
|Mo Sold|mes de venta||
|Yr Sold|año de venta||
|SalePrice|precio de venta en dólares||

Recomiendo al leer los datos, eliminar espacios de los nombres de las columnas, realiza un pequeño análisis inicial de los mismos. No olvides fijarte en los tipos de variables, que variables pueden tener tipos confundidos y corrige los. Sobre todo, trabaja con las fechas.

**NOTA:** Las tareas complementarias sirven para subir nota. El resto de preguntas valen igual y suman 10 puntos.

## Inicializar y cargar el contexto spark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext

In [ ]:
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql import SQLContext, functions as F, Row
from pyspark.sql.types import *
from pyspark.sql.types import StringType, IntegerType

# Librería para dar formato a las salidas obtenidas
import pandas as pd
# Librería para aplicar expresiones regulares en el parseado
import re
# librería para parsear las fechas con formato date
from datetime import datetime
# Librería que permite descargar información de una url desde python
import urllib
from urllib.request import urlretrieve
# Librería para llamar a funciones relacionadas con el sistema operativo
import os

In [ ]:
sc = SparkContext.getOrCreate()

In [ ]:
sqlContext = SQLContext(sc)

## Importamos CSV

In [ ]:
# se importan los datos en formato csv
data = "BDpracticafinalCSV.csv" 
raw_data = sc.textFile(data)

Parseo de datos: se observa que los datos están separados por ";". 

In [ ]:
# separamos en columnas
dt_data= raw_data.map(lambda x: x.split(";")) 

In [ ]:
# visualizamos las dos primeras filas
dt_data.take(2)  

## Cuántas viviendas distintas encontramos en el dataset? ¿Se repite alguna? Tiene sentido que haya duplicadas? ¿Qué podemos hacer con las duplicadas?

Para conocer cuantas viviendas hay y si están duplicadas, primero se van a realizar transformaciones en los datos para que la contabilización sea la correcta.

 - Se elimina la primera fila que es la cabecera, para que no se tenga en cuenta en los análisis

In [ ]:
# identificamos  la cabecera
header = dt_data.take(1)[0] 
header

In [ ]:
# eliminamos la cabecera del rdd de los datos
dt_data_nh = dt_data \
    .filter(lambda line: line!=header)

## Filtramos las zonas residenciales (alta, media y baja densidad)
Se observa que estas zonas contienen diferentes nombres por lo que se decide eliminar aquellos que NO son de estas zonas.
Se eliminan de los datos las ventas de zonas A <- RURAL, C <- COMERCIAL, FV <- RESIDENCIAL FLOTANTE y I <- INDUSTRIAL.

Se seleccionan todas las columnas.


In [ ]:
dt_data_nh_filter = dt_data_nh.map(lambda x: (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13], x[14], x[15], x[16], x[17], x[18], x[19], x[20], x[21], x[22],x[23],x[24],x[25],x[26],x[27],x[28])).\
                filter(lambda x: x[1] != ("A")).\
                filter(lambda x: x[1] != ("C") ).\
                filter(lambda x: x[1] != ("FV")).\
                filter(lambda x: x[1] != ("I") ) 

El rdd dt_data_nh_filter contiene todos los datos de las zonas seleccionadas sin cabecera.
De aquí en adelante todas las operaciones se realizarán sobre la muestra ya filtrada en estas zonas

A continuación conocemos cuantos registros contiene:

In [ ]:
# cuenta del número de filas
(dt_data_nh_filter.map(lambda line: line[0]).count()) 


Hay 2768 registros en los datos.

Se pasa el RDD a formato tabla SQL para poder realizar diferentes cálculos y operaciones.

Hay que tener en cuenta que se realiza con un rdd que no tiene cabecera.

In [ ]:
df_data = sqlContext.createDataFrame (dt_data_nh_filter  )

In [ ]:
# visualizamos el dataframe SQL creado con el rdd
df_data.show(10)

Al no tener cabecera, es importante conocer que el orden es el mismo que el expuesto en metada. Siendo _1 la variable Order y _29 la variable de precio.

In [ ]:
df_data.createOrReplaceTempView("df_data") # necesario para operar con el df

Cuenta de valores únicos en el DF.
Para este cálculo se utiliza la primera columna "Order" que es un identificador único por vivienda:

In [ ]:
sqlContext.sql("""SELECT COUNT(DISTINCT _1) AS Nro_viviendas
                  FROM df_data """).show()

Hay 2768 viviendas únicas y 2762 registros. Esto nos indica que hay viviendas que se encunetran más de una vez en el registro.
Interpretando que tipo de base de datos y qué significa cada uno de los registros he decidido no eliminar los duplicados ya que una sola vivienda puede haber formado parte de dos transacciones diferentes en momentos diferentes, aunque sea en el mismo año. Esto puede aportar posible informacion acerca de la evolución de los precios de mercado.No existe en la metadata más infromación para descartar que una misma vivienda no pueda venderse y registrarse más de una vez y que esto sea una errata. Por lo tanto, se mantienen todos los registros.

## ¿Podrías decirme el total de inmuebles y el precio medio (Sale Price) de cada zona (MS Zoning)?

Mediante SPARK SQL se realiza la siguiente tabla que muestra el conteo de ventas por cada zona y su precio medio. 

Ha hanido que unificar las zonas de las viviendas ya que presentaban formas diferentes como rL y Rl.

In [ ]:
sqlContext.sql("""SELECT  count( _1) AS Nro_viviendas, AVG(_29) AS precio_medio,
                  CASE _2 WHEN 'Rl' THEN 'RL'
                          WHEN 'rL' THEN 'RL'
                          WHEN 'RL' THEN 'RL'
                          WHEN 'RH' THEN 'RH'
                          ELSE 'RM' END AS ZONA
                  FROM df_data 
                  GROUP BY ZONA """).show()

+-------------+------------------+----+
|Nro_viviendas|      precio_medio|ZONA|
+-------------+------------------+----+
|           27|136419.77777777778|  RH|
|         2279| 191412.6068451075|  RL|
|          462|126781.39393939394|  RM|
+-------------+------------------+----+



Como se ha explicado en el ejercicio anterior se decide tener en cuenta todos los registros ya que hay viviendas que pueden haberse vendido en más de una ocasión.

Por lo que esta tabla tabla muestra el total de inmuebles de la BBDD agrupados por zona e indicando su precio medio

## Media de Total Bsmt SF por cada década de construcción calculada a partir de Year Built. 
## ¿Cuál es la decada de construcción con viviendas mejor acondicionadas para el frío (Heating QC)?

El objetivo es agrupar cada transaccion por decadas calculada mediante el campo Year Buit. 

Despues, habrá que utilizar la clasificación Heating QC para identificar cual es la media de este ratio para cada década y así poder establecer un ranking por décadas en función de esta caraterística.

En primer lugar, para realizar la agrupación visualizamos el primer y el último año de construcción de las viviendas:

In [ ]:
sqlContext.sql("""SELECT  min( _8) , max(_8)
                  FROM df_data """).show()

+-------+-------+
|min(_8)|max(_8)|
+-------+-------+
|   1872|   2010|
+-------+-------+



La primera vivienda de la bbdd fue construida en 1872 y la última en 2010.

A continuación mediante SPARK SQL se realiza la agrupación por décadas, indicando en esta nueva columna el siglo y la década a la que pertence cada vivienda.

Además, para poder realizar un ratio acerca de la calidad de de su acondicionamiento al frío se crea otra nueva columna num_heatingQ, que identificará numericamente mediante la siguiente equivalencia: 

Heating QC	Calidad de la calefacción	"Ex Excelente = 5 ,Gd Bueno = 4,TA Media = 3,Fa Justo = 2 ,Po Pobre = 1".

Así las viviendas con mayor calificación numérica serán las mejor acondicionadas al frío.

Esta nueva identificación permite calcula cual es la media de la calidad del acondicionamiento de las viviendas según la década en la que fueron construidas. A mayor media de esta nueva variable en la década de construcción mejoe es el acondicionamiento

In [ ]:
from pyspark.sql.functions import*
df_data.createOrReplaceTempView("df_decade_quality")
df_decade_quality = sqlContext.sql(""" SELECT*, 
CASE
    WHEN _8 < 1880 THEN 'XIX_70s'
    WHEN _8<1890 THEN 'XIX_80s'
    WHEN _8<1900 THEN 'XIX_90s'
    WHEN _8<1910 THEN 'XX_00s'
    WHEN _8<1920 THEN 'XX_10s'
    WHEN _8<1930 THEN 'XX_20s'
    WHEN _8<1940 THEN 'XX_30s'
    WHEN _8<1950 THEN 'XX_40s'
    WHEN _8<1960 THEN 'XX_50s'
    WHEN _8<1970 THEN 'XX_60s'
    WHEN _8<1980 THEN 'XX_70s'
    WHEN _8<1990 THEN 'XX_80s'
    WHEN _8<2000 THEN 'XX_90s'
    WHEN _8<2010 THEN 'XXI_00s'
    ELSE 'XXI_10s'
END  AS decada,
CASE
    WHEN _14 = 'Ex' THEN 5
    WHEN _14 = 'Gd' THEN 4
    WHEN _14 = 'TA' THEN 3
    WHEN _14 = 'Fa' THEN 2
    ELSE 1
END  AS num_heatingQ
FROM df_data 
""").persist()

Se ha relaizado un persist para poder guardar las nuevas columnas creadas. Visualizamos los datos a continuación:

In [ ]:
df_decade_quality.show(10)

+---+---+---+-----+---+---+---+----+----+-------+---+---+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+------+-------+------------+
| _1| _2| _3|   _4| _5| _6| _7|  _8|  _9|    _10|_11|_12| _13|_14|_15|_16|_17|_18|_19|_20|_21|_22|_23|_24|_25|_26|_27| _28|   _29| decada|num_heatingQ|
+---+---+---+-----+---+---+---+----+----+-------+---+---+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+------+-------+------------+
|  1| RL|141|31770|Lvl|Gtl|  6|1960|1960|  Stone| TA| Gd|1080| Fa|  Y|  1|  0|  3|  1| TA|  7|  2|528| TA|  0| Sp|  5|2010|215000| XX_60s|           2|
|  2| RH| 80|11622|Lvl|Gtl|  5|1961|1961|   None| TA| TA| 882| TA|  Y|  1|  0|  2|  1| TA|  5|  1|730| TA|  0| Sp|  6|2010|105000| XX_60s|           3|
|  3| RL| 81|14267|Lvl|Gtl|  6|1958|1958|BrkFace| TA| TA|1329| TA|  Y|  1|  1|  3|  1| Gd|  6|  1|312| TA|  0| Sp|  6|2010|172000| XX_50s|           3|
|  4| RL| 93|11160|Lvl|Gtl|  7|1968|1968|   None| Gd| TA|2110| Ex|  Y|  2|  1|  3|  1| E

In [ ]:
df_decade_quality.createOrReplaceTempView("df_decade_quality")

__Media de tamaño de los sótanos en pies cuadrados por dácada de construcción:__

Una vez tenemos a que década ha pertenecido cada vivienda es posible calcula la media, en pies cuadrados, del tamaño de los sótanos. La variable Total Bsmt SF es la _13 de la tabla de datos.

In [ ]:

sqlContext.sql(""" SELECT decada , AVG(_13) AS media_sotano
 FROM df_decade_quality
 GROUP BY decada
 """).show()

+-------+------------------+
| decada|      media_sotano|
+-------+------------------+
| XX_10s| 788.3009708737864|
| XX_00s| 720.1388888888889|
|XIX_70s|             761.0|
|XIX_80s|             836.0|
| XX_30s| 769.9158878504672|
|XXI_00s|1349.7657657657658|
| XX_90s|            1138.6|
|XIX_90s| 885.5833333333334|
| XX_20s| 833.9368421052632|
| XX_70s|  953.917808219178|
| XX_80s|            1086.0|
| XX_50s| 968.2955223880597|
| XX_60s| 1082.672268907563|
| XX_40s| 706.9261744966443|
|XXI_10s|            1539.0|
+-------+------------------+



__Década con las viviendas mejor acondcionadas al frío:__

Agrupamos los registros por décadas y calculamos la calidad media de sua condicionamiento al frío a traves de la columna numérica calculada.

La década con una media mayor será la mejor acondicionada al frío.

In [ ]:
sqlContext.sql(""" SELECT decada, AVG(num_quality) AS calidad_media
FROM df_decade_quality
GROUP BY decada
ORDER BY calidad_media DESC
""").show()

+-------+------------------+
| decada|     calidad_media|
+-------+------------------+
|XXI_10s|               5.0|
|XXI_00s|4.9624624624624625|
| XX_90s| 4.720634920634921|
|XIX_90s|               4.0|
| XX_00s|3.9444444444444446|
| XX_10s|3.9223300970873787|
| XX_30s|3.9158878504672896|
|XIX_80s|             3.875|
| XX_40s|3.8322147651006713|
| XX_50s|3.7522388059701495|
| XX_80s|              3.75|
| XX_60s|3.6862745098039214|
| XX_20s|3.6473684210526316|
| XX_70s|3.4547945205479453|
|XIX_70s|               3.0|
+-------+------------------+



Se ha ordenado de mayor a menor calidad de acondicionamiento y el resultado muestra que la década con mviviendas mejores acondicionadas al frío es la década que comienza en el 2010. Por el contrario las viviendas peor acondicionadas son las construídas en la decada que comenzó en 1870.

## ¿Cuáles son las 10 viviendas que se vendieron por un precio más elevado por metro cuadrado en el año 2009?

Para calcular estas 10 viviendas se va a utilizar SPARK SQL. 
Hayq ue tener en cuenta que la información del tamaño de la vivienda viene dada en pies cuadrados. Para obtener los metros cuadrados se aplica : 1 pie cudrado = 0.092903 metros cuadrados.

Para conocer el precio de las viviendas por metro cuadrados hay que dividir el valor total de la vivienda entre los metros cuadrados:

Precio_Vivienda/(pies_cudrdados*0.092903)

El precio de la vivienda es la columna _29 y el tamaño en pies cuadrados es la columna _4 

Se añade un orden descente y nos quedamos con los 10 primeros registros:

In [ ]:

sqlContext.sql(""" SELECT _1 AS id_vivienda, _29/(_4*0.092903) AS precio_metros_cuadrados
FROM df_data
WHERE _28 =2009
ORDER BY precio_metros_cuadrados DESC
""").show(10)

+-----------+-----------------------+
|id_vivienda|precio_metros_cuadrados|
+-----------+-----------------------+
|        936|     1026.7118971897073|
|        934|      991.0325013521444|
|        464|      821.0570122815511|
|        935|      807.9613655043873|
|        408|       765.647528938273|
|        407|      756.0368904997173|
|        405|      717.5943367454944|
|        933|       714.281725843034|
|        403|      711.1872444531239|
|        411|      701.0707829388548|
+-----------+-----------------------+
only showing top 10 rows



La vivienda identificada con el número 936 fue la que se vendión por un precio más elevado por metros cuadrados, 1026 dolares por metro cuadrado

## Media anual por zonas del precio de venta y metros cuadrados.

El objetivo es identificar para cada año en el que estamos estudiando las ventas, cual es el precio medio de las viviendas en cada zona.
La bbdd de las ventas comienza en 2006 y finaliza en 2010.

Agrupamos las ventas por años y por zonas, y para cada una de estas combianciones estudiamos su precio medio y su tamaño medio en metros cuadrados.

Se ha procedido a realizar este cálculo mediante SPARK SQL. Hay que tener en cuenta que el año de venta es la columna _28, la zona la columna _2 (sobre la que realizamos la agrupación correcta anteriormente comentada), el precio la columna _29 y el tamaño en metros cuadrados los calculamos _4*0.092903

In [ ]:
sqlContext.sql(""" SELECT _28 AS YEAR, CASE _2 WHEN 'Rl' THEN 'RL'
                          WHEN 'rL' THEN 'RL'
                          WHEN 'RL' THEN 'RL'
                          WHEN 'RH' THEN 'RH'
                          ELSE 'RM' END AS ZONA, AVG(_29) AS precio_medio, AVG(_4*0.092903) AS metros_cuadrados_medios
FROM df_data
GROUP BY _28, ZONA
ORDER BY YEAR
""").show()

+----+----+------------------+-----------------------+
|YEAR|ZONA|      precio_medio|metros_cuadrados_medios|
+----+----+------------------+-----------------------+
|2006|  RL|194911.28194726165|     1043.6885082028389|
|2006|  RM|127502.63725490196|      575.0140103627449|
|2006|  RH|140316.66666666666|      659.5648484999999|
|2007|  RL|193399.98014440434|     1059.6779461064987|
|2007|  RM|128947.62105263158|      625.4484220842105|
|2007|  RH|          147250.0|           651.31970725|
|2008|  RM| 133336.1219512195|      623.6691686341463|
|2008|  RL|189885.58515283844|     1047.6533375414854|
|2008|  RH|150333.33333333334|      766.1400733333334|
|2009|  RM|118759.86904761905|      552.5560763333336|
|2009|  RL|190455.55086372362|     1002.1840689904019|
|2009|  RH|          116206.8|      647.1437173999999|
|2010|  RM|119681.70689655172|      586.1042039137931|
|2010|  RH|          126280.0|            753.0345568|
|2010|  RL|  184978.418972332|     1011.3593166600795|
+----+----

## ¿Podrías decirme el total de recaudación de las casas de revistimiento (Mas Vnr Type) de piedra con respecto a las de ladrillo? ¿Hay diferencia significativa?

El obtivo es identificar la suma total de los precios de las viviendas, analizando si influye que su revestimiento sea piedra o ladrillo.

En primer lugar bisualizamos todos los tipos de revestimiento y la suma de los precios de todas las ventas

In [ ]:
from pyspark.sql.functions import sum
sqlContext.sql("""  SELECT _10 AS material_revestimietno , COUNT(_1) AS num_viviendas,SUM(_29) AS precio_total
FROM df_data
GROUP BY _10
""").show()

+----------------------+-------------+------------+
|material_revestimietno|num_viviendas|precio_total|
+----------------------+-------------+------------+
|                  None|         1646|2.51160498E8|
|                BrkCmn|           25|   3504975.0|
|                 Stone|          230| 6.0495477E7|
|                      |           16|   3831658.0|
|               BrkFace|          850|1.79413061E8|
|                CBlock|            1|     80000.0|
+----------------------+-------------+------------+




Hay que tener en cuenta que de ladrillo hay 875 ventas en total mientras que de piedra hay 230. Para poder hacer un cáculo en el que se puedan comparar mejor ambos recubrimientos calcularemos el precio medio para las de ladrillo y el precio medio para las de piedra.


Realizamos estos cáculos mediante SPARK SQL agrupando las casas por los carácteres contenidos en su tipo de recubriemietno,
Se observa que aquella que están revestidas con ladrillo tienen en común la parte Brk.
Y las de piedra Stone.


In [ ]:
from pyspark.sql.functions import sum
sqlContext.sql("""  SELECT AVG(_29) AS precio_medio_ladrillo
FROM df_data
WHERE _10 LIKE '%Brk%'
""").show()

+---------------------+
|precio_medio_ladrillo|
+---------------------+
|           209049.184|
+---------------------+



In [ ]:
from pyspark.sql.functions import sum
sqlContext.sql("""  SELECT AVG(_29) AS precio_medio_piedra
FROM df_data
WHERE _10 LIKE '%Stone%'
""").show()

+-------------------+
|precio_medio_piedra|
+-------------------+
| 263023.81304347824|
+-------------------+



Se observa que la recaudación media por vivienda es mayor para las recubiertas por piedra. Sin embargo, al haber más casas de ladrillo la recudación total es mayor la de las casas recubiertas de ladrillo

## ¿Cuánto son más caras las viviendas con 2 cocinas, con 2 o más plazas de garaje que las que tienen 1 cocina y 1 plaza de garaje? Comparar medias y cuartiles de ambos casos

El objetivo es conocer las estadísticas de las vivivendas con las caracterísiticas señaladas.


__Viendas con dos cocinas y dos o más plazas de garage.__

En primer lugar, se va a estudiar la distribución y los valores medios de los precios para viviedas con dos cocinas y dos o más plazas de garages.
Para ello se va trabajar con los rdd. Nos interesa filtrar aquellas viviendas con dos cocinas, el número de cocinas es la columna [18], y con dos o más garges, columna [21]. Además necesitamos conocer el precio de estas viviendas

In [ ]:
dt_2coci_2bath = dt_data_nh_filter.map(lambda x: (x[0],x[18],x[21],x[28])).\
                filter(lambda x: x[1] == '2').\
                filter(lambda x: x[2] >'1' )

In [ ]:
dt_2coci_2bath.take(10) # visualizamos que ha cogido las características correctas

In [ ]:
# indicamos el tipo de columnas con el que estamos trabajando ya que necesimaos hacer algunos cálculos

In [ ]:
dt_2coci_2bath_clean = dt_2coci_2bath.map(lambda x: (str(x[0]),  float(x[1]), float(x[2]), float(x[3]) ))

In [ ]:
# se crea el df para poder extraer las carácterisicas numéricas

In [ ]:
df_2coci_2bath_clean = sqlContext.createDataFrame (dt_2coci_2bath_clean  )

Caracterísiticas numéricas: nos interesa las de la columna _4 ya que nos indica el precio

In [ ]:
df_2coci_2bath_clean.summary().show()

Para viviendas con dos cocinas y dos o más plazas de garage el precio medio es 145124.28 dollars. En cuanto a los cuartiles, el cuartil 25% es 118964$, esto quiere decir que el 25% de las vivivendas de estas caracterísiticas su precio de venta fue menor que 118964. La mediana es 141000. Y el cuartil 75% es 159000, esto indica que el 75% de las viviendas de estas caracterísitcas  se vendieron por menos de 159000 dólares.

Comprobamos que la media es igual que calculandolo sobre el rdd

In [ ]:
dt_2coci_2bath_clean.map(lambda x: (x[3])).mean()

145124.27906976745

Realizamos el mismo porceso para:

__Viviendas con una cocina y una plaza de garage:__

Como se ha realiza anteriormente filtramos el rdd para que tenga las caraterísticas que se han indicado

In [ ]:
dt_1coci_1gara = dt_data_nh_filter.map(lambda x: (x[0],x[18],x[21],x[28])).\
                filter(lambda x: x[1] == '1').\
                filter(lambda x: x[2] =='1' )

Indicamos el tipo de columnas con el que estamos trabajando para poder realizar los cálculos sobre ellas:

In [ ]:
dt_1coci_1gara_clean = dt_1coci_1gara.map(lambda x: (str(x[0]),  float(x[1]), float(x[2]), float(x[3]) ))

In [ ]:
dt_1coci_1gara.take(10)

[('2', '1', '1', '105000'),
 ('3', '1', '1', '172000'),
 ('26', '1', '1', '142000'),
 ('30', '1', '1', '96000'),
 ('31', '1', '1', '105500'),
 ('32', '1', '1', '88000'),
 ('35', '1', '1', '120000'),
 ('76', '1', '1', '143000'),
 ('78', '1', '1', '99500'),
 ('79', '1', '1', '125000')]

Se crea un data frame para poder realizar las medidas estadísitcas de las variables

In [ ]:
df_1coci_1gara = sqlContext.createDataFrame (dt_1coci_1gara  )

Visualizamos las medidas estadísticas. La variable precio es la columna_4, con las otras dos columnas podemos comprobar que la muestra seleccionada cumple con los reqisitos

In [ ]:
df_1coci_1gara.summary().show()

+-------+------------------+---+---+------------------+
|summary|                _1| _2| _3|                _4|
+-------+------------------+---+---+------------------+
|  count|               755|755|755|               755|
|   mean|1515.2503311258279|1.0|1.0|128121.99337748345|
| stddev| 844.3682739651009|0.0|0.0|30303.813376544425|
|    min|              1036|  1|  1|            100000|
|    25%|             759.0|1.0|1.0|          111500.0|
|    50%|            1489.0|1.0|1.0|          128000.0|
|    75%|            2202.0|1.0|1.0|          142125.0|
|    max|               982|  1|  1|             99900|
+-------+------------------+---+---+------------------+



Para viviendas con una cocina y una plaza de garage el precio medio es 128122 dollars. En cuanto a los cuartiles, el cuartil 25% es 111500$, esto quiere decir que el 25% de las vivivendas de estas caracterísiticas su precio de venta fue menor que 111500. La mediana es 128000. Y el cuartil 75% es 142125, esto indica que el 75% de las viviendas de estas caracterísitcas  se vendieron por menos de 142125 dólares.

Comprobamos que la media es igual que calculandolo sobre el rdd

In [ ]:
dt_1coci_1gara_clean.map(lambda x: (x[3])).mean()

128121.9933774834

In [9]:

145124.27906976745 - 128121.9933774834 # diferencia en precios medios




17002.285692284044

In [ ]:
141000 - 128000.0 # diferencia en las medianas

13000.0

__COMPARACIÓN__

Analizando la media de los precios de las viviendas analizadas se observa que  la viviendas con dos cocionas y dos o más plazas de garage son en promedio 17002 dólares más caras. La mediana del primer tipo de viviendas es  13000 más que las del primer tipo. Estas diferencias se observan también en los valores de los cuartiles. Con esto, se puede concluir que al aumentar el número de cocinas de una a dos y de la pzadas de garage a más de una, supone un incremento en el precio medio de las viviendas.

## (COMPLEMENTARIA) Estudiar la relación entre el precio y el número de Garajes. (Recomiendo segmentar precio por cuartiles y estudiar el número de casos coincidentes)

Se desea conocer la posible correlación entre el precio y el número de garages. Ya que no se especifica la técnica a seguir se decide realizarlo de una manera simple que nos puede dar una idea de la exisitencia de correlaciones positivas o negativas para una fácil interpretación.
Se realiza mediante una fórmula de SPARK SQ, en la que la variable dependiente es el precio y la explicativa será el número de plazas de garages de la vivienda

In [ ]:
sqlContext.sql("""SELECT CORR(_29,_22) AS correlacion_precio_ngarages
                  FROM df_data """).show()

+---------------------------+
|correlacion_precio_ngarages|
+---------------------------+
|         0.6460054684088166|
+---------------------------+



Se puede realizar una sencilla interpretación del resultado. Existe una correlación positiva entre el número de plazas de garage y el precio. Cuando las plazas de garage aumentan, el precio también aumenta.

## (COMPLEMENTARIA) Las 10 viviendas con mejores servicios y mejor precio.

### Pista:
#### Calcula las variables: 
- Número de servicios excelentes
- Número de servicios buenos
...

#### Tendréis que tener en cuenta también variables como número de baños Full Bath, cocinas Kitchen AbvGr o dormitorios Bedroom AbvGr. Generando por ejemplo (número de estas variables por encima de media)

#### Finalmente precio de venta

## (COMPLEMENTARIA) Importar los datos en formato EXCEL

A continuación se han importado los datos en excel para trabajar con ellos en un formato data frame en el que poder trabajar con SPARK

In [ ]:
data_excel = "BDpracticafinal2021.xlsx"

Importamos los datos nombrados y los visualizamos

In [ ]:
pdf = pd.read_excel(data_excel)
pdf

,Order,MS Zoning,Lot Frontage,Lot Area,Land Contour,Land Slope,Overall Qual,Year Built,Year Remod/Add,Mas Vnr Type,Exter Qual,Bsmt Cond,Total Bsmt SF,Heating QC,Central Air,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Garage Cars,Garage Area,Garage Cond,Pool Area,Pool QC,Mo Sold,Yr Sold,SalePrice
0,1,RL,141.0,31770,Lvl,Gtl,6,1960,1960,Stone,TA,Gd,1080,Fa,Y,1,0,3,1,TA,7,2.0,528.0,TA,0,Sp,5,2010,215000
1,2,RH,80.0,11622,Lvl,Gtl,5,1961,1961,None,TA,TA,882,TA,Y,1,0,2,1,TA,5,1.0,730.0,TA,0,Sp,6,2010,105000
2,3,RL,81.0,14267,Lvl,Gtl,6,1958,1958,BrkFace,TA,TA,1329,TA,Y,1,1,3,1,Gd,6,1.0,312.0,TA,0,Sp,6,2010,172000
3,4,RL,93.0,11160,Lvl,Gtl,7,1968,1968,None,Gd,TA,2110,Ex,Y,2,1,3,1,Ex,8,2.0,522.0,TA,0,Sp,4,2010,244000
4,5,RL,74.0,13830,Lvl,Gtl,5,1997,1998,None,TA,TA,928,Gd,Y,2,1,3,1,TA,6,2.0,482.0,TA,0,Sp,3,2010,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2930,RL,74.0,9627,Lvl,Mod,7,1993,1994,BrkFace,TA,TA,996,Ex,Y,2,1,3,1,TA,9,3.0,650.0,TA,0,Sp,11,2006,188000
2932,2898,Rl,NaN,11170,Lvl,Gtl,7,1990,1991,None,TA,TA,1216,Ex,Y,2,1,4,1,TA,8,2.0,693.0,TA,0,Sp,4,2006,250000
2933,2899,Rl,NaN,8098,Lvl,Gtl,6,2000,2000,None,Gd,TA,1381,Ex,Y,2,0,2,1,Gd,5,2.0,470.0,TA,0,Sp,10,2006,202000
2934,2900,Rl,85.0,14331,Lvl,Gtl,8,2002,2002,BrkFace,Gd,TA,1800,Ex,Y,2,0,3,1,Gd,7,3.0,765.0,TA,0,Sp,5,2006,312500


Se observa que actualmente tienen forma de dataframe de python, por lo que es necesario realizar más transformaciones para porde leerlos con spark.

Es necesario importar el paquete que habilita la lectura de datos en excel:

In [ ]:
spark = SparkSession.builder \
.master("local") \
.appName("Word Count") \
.config("spark.jars.packages", "com.crealytics:spark-excel_2.11:0.12.2") \
.getOrCreate()

Definimos la estructura de los datos que vamos a importar. Hay que indicar que tipo de columna es cada una para que más tarde se pueda leer el excel correctamente y formar un data frame en el que se puedan realizar los cálculos en SPARK

In [ ]:
from pyspark.sql.types import *

mySchema = StructType([ StructField("col1", StringType(), True)\
                       ,StructField("col2", StringType(), True)\
                       ,StructField("col3", FloatType(), True)\
                       ,StructField("col4", IntegerType(), True)\
                       ,StructField("col5", StringType(), True)\
                       ,StructField("col6", StringType(), True)\
                       ,StructField("col7", IntegerType(), True)\
                       ,StructField("col8", IntegerType(), True)\
                       ,StructField("col9", IntegerType(), True)\
                       ,StructField("col10", StringType(), True)\
                       ,StructField("col11", StringType(), True)\
                       ,StructField("col12", StringType(), True)\
                       ,StructField("col13", IntegerType(), True)\
                       ,StructField("col14", StringType(), True)\
                       ,StructField("col15", StringType(), True)\
                       ,StructField("col16", IntegerType(), True)\
                       ,StructField("col17", IntegerType(), True)\
                       ,StructField("col18", IntegerType(), True)\
                       ,StructField("col19", IntegerType(), True)\
                       ,StructField("col20", StringType(), True)\
                       ,StructField("col21", IntegerType(), True)\
                       ,StructField("col22", FloatType(), True)\
                       ,StructField("col23", FloatType(), True)\
                       ,StructField("col24", StringType(), True)\
                       ,StructField("col25", IntegerType(), True)\
                       ,StructField("col26", StringType(), True)\
                       ,StructField("col27", IntegerType(), True)\
                       ,StructField("col28", IntegerType(), True)\
                       ,StructField("col29", IntegerType(), True)
                       ])

Aplicamos al data frame de python que hemos creado el formato de datos que hemos indicado en el esquema

In [ ]:
df = spark.createDataFrame(pdf,schema=mySchema)

Se muestra el data frame creado en SPARK

In [ ]:
df.show()

+----+----+-----+-----+----+----+----+----+----+-------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
|col1|col2| col3| col4|col5|col6|col7|col8|col9|  col10|col11|col12|col13|col14|col15|col16|col17|col18|col19|col20|col21|col22|col23|col24|col25|col26|col27|col28| col29|
+----+----+-----+-----+----+----+----+----+----+-------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
|   1|  RL|141.0|31770| Lvl| Gtl|   6|1960|1960|  Stone|   TA|   Gd| 1080|   Fa|    Y|    1|    0|    3|    1|   TA|    7|  2.0|528.0|   TA|    0|   Sp|    5| 2010|215000|
|   2|  RH| 80.0|11622| Lvl| Gtl|   5|1961|1961|   None|   TA|   TA|  882|   TA|    Y|    1|    0|    2|    1|   TA|    5|  1.0|730.0|   TA|    0|   Sp|    6| 2010|105000|
|   3|  RL| 81.0|14267| Lvl| Gtl|   6|1958|1958|BrkFace|   TA|   TA| 1329|   TA|    Y|    1|    1|    3|    1|   Gd|    6|  1.0|312.0|   TA|